In [1]:
import os
import shutil
import pandas as pd

# RAW extracted from step 6
RAW_IMG_DIR = "/home/kakani/waymo_project/1_data/extracted/images_raw"
RAW_LBL_DIR = "/home/kakani/waymo_project/1_data/extracted/labels_raw"

# Final YOLO dataset folders
YOLO_IMG_TRAIN = "/home/kakani/waymo_project/2_yolo_dataset/images/train"
YOLO_LBL_TRAIN = "/home/kakani/waymo_project/2_yolo_dataset/labels/train"

YOLO_IMG_VAL = "/home/kakani/waymo_project/2_yolo_dataset/images/val"
YOLO_LBL_VAL = "/home/kakani/waymo_project/2_yolo_dataset/labels/val"

print("RAW IMG:", RAW_IMG_DIR)
print("RAW LBL:", RAW_LBL_DIR)

print("\nYOLO TRAIN IMG:", YOLO_IMG_TRAIN)
print("YOLO TRAIN LBL:", YOLO_LBL_TRAIN)
print("\nYOLO VAL IMG:", YOLO_IMG_VAL)
print("YOLO VAL LBL:", YOLO_LBL_VAL)


RAW IMG: /home/kakani/waymo_project/1_data/extracted/images_raw
RAW LBL: /home/kakani/waymo_project/1_data/extracted/labels_raw

YOLO TRAIN IMG: /home/kakani/waymo_project/2_yolo_dataset/images/train
YOLO TRAIN LBL: /home/kakani/waymo_project/2_yolo_dataset/labels/train

YOLO VAL IMG: /home/kakani/waymo_project/2_yolo_dataset/images/val
YOLO VAL LBL: /home/kakani/waymo_project/2_yolo_dataset/labels/val


In [2]:
raw_images = sorted([f for f in os.listdir(RAW_IMG_DIR) if f.endswith(".jpg")])
raw_labels = sorted([f for f in os.listdir(RAW_LBL_DIR) if f.endswith(".txt")])

print("Total RAW images:", len(raw_images))
print("Total RAW labels:", len(raw_labels))

print("\nSample RAW images:", raw_images[:5])
print("Sample RAW labels:", raw_labels[:5])


Total RAW images: 50
Total RAW labels: 50

Sample RAW images: ['train_frame_0000.jpg', 'train_frame_0001.jpg', 'train_frame_0002.jpg', 'train_frame_0003.jpg', 'train_frame_0004.jpg']
Sample RAW labels: ['train_frame_0000.txt', 'train_frame_0001.txt', 'train_frame_0002.txt', 'train_frame_0003.txt', 'train_frame_0004.txt']


In [3]:
split_ratio = 0.8
split_index = int(len(raw_images) * split_ratio)

train_images = raw_images[:split_index]
val_images   = raw_images[split_index:]

print("Train images:", len(train_images))
print("Val images:", len(val_images))


Train images: 40
Val images: 10


In [4]:
# YOLO class IDs
# 0: VEHICLE
# 1: PEDESTRIAN
# 2: SIGN
# 3: CYCLIST

waymo_to_yolo = {
    "VEHICLE": 0,
    "PEDESTRIAN": 1,
    "SIGN": 2,
    "CYCLIST": 3
}

print("Class map:", waymo_to_yolo)


Class map: {'VEHICLE': 0, 'PEDESTRIAN': 1, 'SIGN': 2, 'CYCLIST': 3}


In [5]:
from PIL import Image

def convert_raw_to_yolo(raw_label_path, image_path, yolo_label_path):
    img = Image.open(image_path)
    W, H = img.size

    lines_out = []

    with open(raw_label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        cls_name, xmin, ymin, xmax, ymax = parts
        cls_name = cls_name.upper()

        if cls_name not in waymo_to_yolo:
            continue

        cls_id = waymo_to_yolo[cls_name]

        xmin, ymin, xmax, ymax = map(float, [xmin, ymin, xmax, ymax])

        # convert
        x_center = ((xmin + xmax) / 2) / W
        y_center = ((ymin + ymax) / 2) / H
        w = (xmax - xmin) / W
        h = (ymax - ymin) / H

        lines_out.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    with open(yolo_label_path, "w") as f:
        f.write("\n".join(lines_out))


In [6]:
def process_split(image_list, yolo_img_dir, yolo_lbl_dir):
    for img_file in image_list:
        base = img_file.replace(".jpg", "")

        raw_img_path = os.path.join(RAW_IMG_DIR, img_file)
        raw_lbl_path = os.path.join(RAW_LBL_DIR, base + ".txt")

        yolo_img_path = os.path.join(yolo_img_dir, img_file)
        yolo_lbl_path = os.path.join(yolo_lbl_dir, base + ".txt")

        # copy image
        shutil.copy(raw_img_path, yolo_img_path)

        # convert label
        if os.path.exists(raw_lbl_path):
            convert_raw_to_yolo(raw_lbl_path, raw_img_path, yolo_lbl_path)
        else:
            open(yolo_lbl_path, "w").close()  # empty label file

print("✅ Ready to process train/val splits")


✅ Ready to process train/val splits


In [7]:
process_split(train_images, YOLO_IMG_TRAIN, YOLO_LBL_TRAIN)
process_split(val_images, YOLO_IMG_VAL, YOLO_LBL_VAL)

print("✅ Step 7 Completed: RAW → YOLO conversion done!")


✅ Step 7 Completed: RAW → YOLO conversion done!


In [8]:
train_img_count = len([f for f in os.listdir(YOLO_IMG_TRAIN) if f.endswith(".jpg")])
train_lbl_count = len([f for f in os.listdir(YOLO_LBL_TRAIN) if f.endswith(".txt")])

val_img_count = len([f for f in os.listdir(YOLO_IMG_VAL) if f.endswith(".jpg")])
val_lbl_count = len([f for f in os.listdir(YOLO_LBL_VAL) if f.endswith(".txt")])

print("TRAIN images:", train_img_count, "| labels:", train_lbl_count)
print("VAL   images:", val_img_count, "| labels:", val_lbl_count)


TRAIN images: 40 | labels: 40
VAL   images: 10 | labels: 10


In [9]:
sample = train_images[0].replace(".jpg", ".txt")
path = os.path.join(YOLO_LBL_TRAIN, sample)

print("Sample label file:", sample)
print(open(path).read())


Sample label file: train_frame_0000.txt

